<a target="_blank" href="https://colab.research.google.com/github/ricocf/Leaflet-Product-Classification/blob/main/text_model/text_model.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# **Library Import and Setup**

In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import pandas as pd
import re
import joblib
import pickle
import tensorflow
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
!wget https://github.com/ricocf/Leaflet-Product-Classification/raw/main/data/extracted_text_data/OCR_PSM11_grey_train.csv
!wget https://github.com/ricocf/Leaflet-Product-Classification/raw/main/data/extracted_text_data/OCR_PSM11_rgb_train.csv
!wget https://github.com/ricocf/Leaflet-Product-Classification/raw/main/data/extracted_text_data/OCR_PSM3_grey_train.csv
!wget https://github.com/ricocf/Leaflet-Product-Classification/raw/main/data/extracted_text_data/OCR_PSM3_rgb_train.csv
!wget https://github.com/ricocf/Leaflet-Product-Classification/raw/main/data/extracted_text_data/OCR_pattern_PSM11_rgb_test.xlsx
!wget https://github.com/ricocf/Leaflet-Product-Classification/raw/main/data/extracted_text_data/OCR_pattern_PSM11_rgb_train.xlsx
!wget https://github.com/ricocf/Leaflet-Product-Classification/raw/main/data/extracted_text_data/OCR_pattern_PSM12_rgb_test.xlsx
!wget https://github.com/ricocf/Leaflet-Product-Classification/raw/main/data/extracted_text_data/OCR_pattern_PSM12_rgb_train.xlsx
!wget https://github.com/ricocf/Leaflet-Product-Classification/raw/main/data/extracted_text_data/OCR_pattern_PSM3_rgb_test.xlsx
!wget https://github.com/ricocf/Leaflet-Product-Classification/raw/main/data/extracted_text_data/OCR_pattern_PSM3_rgb_train.xlsx
!wget https://github.com/ricocf/Leaflet-Product-Classification/raw/main/data/extracted_text_data/OCR_pattern_PSM6_rgb_test.xlsx
!wget https://github.com/ricocf/Leaflet-Product-Classification/raw/main/data/extracted_text_data/OCR_pattern_PSM6_rgb_train.xlsx
!wget https://github.com/ricocf/Leaflet-Product-Classification/raw/main/data/extracted_text_data/OCR_test_combined.csv

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


--2024-02-18 18:33:53--  https://github.com/ricocf/Leaflet-Product-Classification/raw/main/data/extracted_text_data/OCR_PSM11_grey_train.csv
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ricocf/Leaflet-Product-Classification/main/data/extracted_text_data/OCR_PSM11_grey_train.csv [following]
--2024-02-18 18:33:54--  https://raw.githubusercontent.com/ricocf/Leaflet-Product-Classification/main/data/extracted_text_data/OCR_PSM11_grey_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4306328 (4.1M) [text/plain]
Saving to: ‘OCR_PSM11_grey_train.csv’

OCR_PSM11_grey_trai 100%[================

# **Data Loading and Preparation**

```
Data Preprocessing for Training Data
Data Preprocessing for Merging Training Data
Data Preprocessing for Merging Test Data
Final Merging of Test Data
Final Merging of Output Data
```

In [2]:
#Data Preprocessing for Training Data:
df = pd.read_excel('/content/OCR_pattern_PSM3_rgb_train.xlsx')
columns_to_combine = ['Pattern 1 Result', 'Pattern 2 Result', 'Pattern 3 Result','Pattern 4 Result','Pattern 5 Result','Pattern 6 Result','Pattern 7 Result']  # Replace with your column names
combined_column = df[columns_to_combine].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
df['combined_column'] = combined_column
df.drop(columns=columns_to_combine, inplace=True)
df.to_excel('/content/OCR_pattern_PSM3_rgb_train.xlsx', index=False)

df = pd.read_excel('/content/OCR_pattern_PSM11_rgb_train.xlsx')
columns_to_combine = ['Pattern 1 Result', 'Pattern 2 Result', 'Pattern 3 Result','Pattern 4 Result','Pattern 5 Result','Pattern 6 Result','Pattern 7 Result']  # Replace with your column names
combined_column = df[columns_to_combine].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
df['combined_column'] = combined_column
df.drop(columns=columns_to_combine, inplace=True)
df.to_excel('/content/OCR_pattern_PSM11_rgb_train.xlsx', index=False)

df = pd.read_excel('/content/OCR_pattern_PSM6_rgb_train.xlsx')
columns_to_combine = ['Pattern 1 Result', 'Pattern 2 Result', 'Pattern 3 Result','Pattern 4 Result','Pattern 5 Result','Pattern 6 Result','Pattern 7 Result']  # Replace with your column names
combined_column = df[columns_to_combine].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
df['combined_column'] = combined_column
df.drop(columns=columns_to_combine, inplace=True)
df.to_excel('/content/OCR_pattern_PSM6_rgb_train.xlsx', index=False)

excel_file_path1 = '/content/OCR_pattern_PSM3_rgb_train.xlsx'
df1 = pd.read_excel(excel_file_path1)
excel_file_path2 = '/content/OCR_pattern_PSM12_rgb_train.xlsx'
df2 = pd.read_excel(excel_file_path2)
merged_df = pd.merge(df1, df2, on='Image File', how='outer', suffixes=('_left', '_right'))
merged_df = merged_df.replace('\n', ' ', regex=True)
excel_file_path = 'output_1.xlsx'
merged_df.to_excel(excel_file_path, index=False)
print(f"Combined data saved to {excel_file_path}")

#Data Preprocessing for Merging Training Data:
excel_file_path1 = 'output_1.xlsx'
df1 = pd.read_excel(excel_file_path1)
excel_file_path2 = '/content/OCR_pattern_PSM11_rgb_train.xlsx'
df2 = pd.read_excel(excel_file_path2)
merged_df = pd.merge(df1, df2, on='Image File', how='outer', suffixes=('_left', '_right'))
merged_df = merged_df.replace('\n', ' ', regex=True)
excel_file_path = 'output_1.xlsx'
merged_df.to_excel(excel_file_path, index=False)
print(f"Combined data saved to {excel_file_path}")

excel_file_path1 = 'output_1.xlsx'
df1 = pd.read_excel(excel_file_path1)
excel_file_path2 = '/content/OCR_pattern_PSM6_rgb_train.xlsx'
df2 = pd.read_excel(excel_file_path2)
merged_df = pd.merge(df1, df2, on='Image File', how='outer')
merged_df = merged_df.replace('\n', ' ', regex=True)
excel_file_path = 'output_1.xlsx'
merged_df.to_excel(excel_file_path, index=False)
print(f"Combined data saved to {excel_file_path}")
#merged_df.head(100)

#Data Preprocessing for Merging Test Data:
df = pd.read_excel('/content/OCR_pattern_PSM12_rgb_test.xlsx')
columns_to_combine = ['Pattern 1 Result', 'Pattern 2 Result', 'Pattern 3 Result','Pattern 4 Result','Pattern 5 Result','Pattern 6 Result','Pattern 7 Result']  # Replace with your column names
combined_column = df[columns_to_combine].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
df['combined_column'] = combined_column
df.drop(columns=columns_to_combine, inplace=True)
df.to_excel('/content/OCR_pattern_PSM12_rgb_test.xlsx', index=False)

df = pd.read_excel('/content/OCR_pattern_PSM11_rgb_test.xlsx')
columns_to_combine = ['Pattern 1 Result', 'Pattern 2 Result', 'Pattern 3 Result','Pattern 4 Result','Pattern 5 Result','Pattern 6 Result','Pattern 7 Result']  # Replace with your column names
combined_column = df[columns_to_combine].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
df['combined_column'] = combined_column
df.drop(columns=columns_to_combine, inplace=True)
df.to_excel('/content/OCR_pattern_PSM11_rgb_test.xlsx', index=False)

df = pd.read_excel('/content/OCR_pattern_PSM6_rgb_test.xlsx')
columns_to_combine = ['Pattern 1 Result', 'Pattern 2 Result', 'Pattern 3 Result','Pattern 4 Result','Pattern 5 Result','Pattern 6 Result','Pattern 7 Result']  # Replace with your column names
combined_column = df[columns_to_combine].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
df['combined_column'] = combined_column
df.drop(columns=columns_to_combine, inplace=True)
df.to_excel('/content/OCR_pattern_PSM6_rgb_test.xlsx', index=False)

df = pd.read_excel('/content/OCR_pattern_PSM3_rgb_test.xlsx')
columns_to_combine = ['Pattern 1 Result', 'Pattern 2 Result', 'Pattern 3 Result','Pattern 4 Result','Pattern 5 Result','Pattern 6 Result','Pattern 7 Result']  # Replace with your column names
combined_column = df[columns_to_combine].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
df['combined_column'] = combined_column
df.drop(columns=columns_to_combine, inplace=True)
df.to_excel('/content/OCR_pattern_PSM3_rgb_test.xlsx', index=False)

#Final Merging of Test Data
excel_file_path1 = '/content/OCR_pattern_PSM12_rgb_test.xlsx'
df1 = pd.read_excel(excel_file_path1)
excel_file_path2 = '/content/OCR_pattern_PSM11_rgb_test.xlsx'
df2 = pd.read_excel(excel_file_path2)
merged_df = pd.merge(df1, df2, on='Image File', how='outer', suffixes=('_left', '_right'))
merged_df = merged_df.replace('\n', ' ', regex=True)
excel_file_path = 'output_1.xlsx'
merged_df.to_excel(excel_file_path, index=False)
print(f"Combined data saved to {excel_file_path}")

excel_file_path1 = '/content/OCR_pattern_PSM3_rgb_test.xlsx'
df1 = pd.read_excel(excel_file_path1)
excel_file_path2 = '/content/OCR_pattern_PSM6_rgb_test.xlsx'
df2 = pd.read_excel(excel_file_path2)
merged_df = pd.merge(df1, df2, on='Image File', how='outer', suffixes=('_left', '_right'))
merged_df = merged_df.replace('\n', ' ', regex=True)
excel_file_path = 'output_2.xlsx'
merged_df.to_excel(excel_file_path, index=False)
print(f"Combined data saved to {excel_file_path}")


Combined data saved to output_1.xlsx
Combined data saved to output_1.xlsx
Combined data saved to output_1.xlsx
Combined data saved to output_1.xlsx
Combined data saved to output_2.xlsx


In [3]:
#Final Merging of Output Data
excel_file_path1 = '/content/output_1.xlsx'
df1 = pd.read_excel(excel_file_path1)
excel_file_path2 = '/content/output_2.xlsx'
df2 = pd.read_excel(excel_file_path2)
merged_df = pd.merge(df1, df2, on='Image File', how='outer', suffixes=('_left', '_right'))
merged_df = merged_df.replace('\n', ' ', regex=True)
excel_file_path = 'output_2.xlsx'
merged_df.to_excel(excel_file_path, index=False)
print(f"Combined data saved to {excel_file_path}")
merged_df.head()


excel_file_path = '/content/output_2.xlsx'
df = pd.read_excel(excel_file_path)
def combine_columns(row):
    return f"{row['combined_column_left_left']} {row['combined_column_right_left']} {row['combined_column_left_right']} {row['combined_column_right_right']}"
df['combined_column'] = df.apply(combine_columns, axis=1)
new_csv_file_path = 'quantity_test.csv'
df.to_csv(new_csv_file_path, index=False)
print(f"Columns combined. Modified data saved to {new_csv_file_path}")

csv_file_path = 'quantity_test.csv'
df = pd.read_csv(csv_file_path)
columns_to_delete = ['combined_column_left_left', 'combined_column_right_left', 'combined_column_left_right', 'combined_column_right_right','Class Name_left_left','Class Name_left_right','Class Name_right_left','Class Name_right_right']
df.drop(columns=columns_to_delete, inplace=True)
new_csv_file_path = 'quantity_test.csv'
df.to_csv(new_csv_file_path, index=False)
print(f"Columns {columns_to_delete} deleted. Modified data saved to {new_csv_file_path}")
df.head()


Combined data saved to output_2.xlsx
Columns combined. Modified data saved to quantity_test.csv
Columns ['combined_column_left_left', 'combined_column_right_left', 'combined_column_left_right', 'combined_column_right_right', 'Class Name_left_left', 'Class Name_left_right', 'Class Name_right_left', 'Class Name_right_right'] deleted. Modified data saved to quantity_test.csv


,Image File,combined_column
0,38319.jpg,75ml 100 ml 75ml 100 ml 75ml 75ml
1,38312.jpg,nan nan nan nan
2,38315.jpg,5 ml 1100 ml 5 ml 1100 ml 5 ml 1100 ml 15 W
3,38314.jpg,75 ml nan 75 ml nan
4,38316.jpg,nan nan nan nan


# **Data Processing for output**

In [4]:
#Data Processing for output_1.xlsx and Conversion to CSV
excel_file_path = '/content/output_1.xlsx'
df = pd.read_excel(excel_file_path)
def combine_columns(row):
    return f"{row['combined_column_left']} {row['combined_column_right']} {row['combined_column_left']} {row['combined_column_right']}"
df['combined_column'] = df.apply(combine_columns, axis=1)
new_csv_file_path = 'quantity_train.csv'
df.to_csv(new_csv_file_path, index=False)
print(f"Columns combined. Modified data saved to {new_csv_file_path}")
df.head()

csv_file_path = 'quantity_train.csv'
df = pd.read_csv(csv_file_path)
columns_to_delete = ['combined_column_left', 'combined_column_right', 'Class Name_left']
df.drop(columns=columns_to_delete, inplace=True)
new_csv_file_path = 'quantity_train.csv'
df.to_csv(new_csv_file_path, index=False)
print(f"Columns {columns_to_delete} deleted. Modified data saved to {new_csv_file_path}")
df.head(100)

#Data Processing for output_2.csv
csv_file_path1 = '/content/OCR_PSM11_grey_train.csv'
df1 = pd.read_csv(csv_file_path1,encoding='latin1')
csv_file_path2 = '/content/OCR_PSM11_rgb_train.csv'
df2 = pd.read_csv(csv_file_path2,encoding='latin1')
merged_df = pd.merge(df1, df2, on='Image File', how='outer', suffixes=('_left', '_right'))
merged_df = merged_df.replace('\n', ' ', regex=True)
csv_file_path = 'output_1.csv'
merged_df.to_csv(csv_file_path, index=False)
print(f"Combined data saved to {csv_file_path}")

csv_file_path1 = '/content/OCR_PSM3_rgb_train.csv'
df1 = pd.read_csv(csv_file_path1,encoding='latin1')
csv_file_path2 = '/content/OCR_PSM3_grey_train.csv'
df2 = pd.read_csv(csv_file_path2,encoding='latin1')
merged_df = pd.merge(df1, df2, on='Image File', how='outer', suffixes=('_left', '_right'))
merged_df = merged_df.replace('\n', ' ', regex=True)
csv_file_path = 'output_2.csv'
merged_df.to_csv(csv_file_path, index=False)
print(f"Combined data saved to {csv_file_path}")
df.head()

Columns combined. Modified data saved to quantity_train.csv
Columns ['combined_column_left', 'combined_column_right', 'Class Name_left'] deleted. Modified data saved to quantity_train.csv
Combined data saved to output_1.csv
Combined data saved to output_2.csv


**Column Deletion and Merging output_1.csv and output_2.csv to ocr_result.csv**

In [5]:
#Column Deletion for output_1.csv:
csv_file_path = '/content/output_1.csv'
df = pd.read_csv(csv_file_path)
column_to_delete = 'Class Name_right'
if column_to_delete in df.columns:
    df = df.drop(columns=[column_to_delete])
    new_csv_file_path = 'output_1.csv'
    df.to_csv(new_csv_file_path, index=False)
    print(f"Column '{column_to_delete}' deleted. New data saved to {new_csv_file_path}")
else:
    print(f"Column '{column_to_delete}' does not exist in the DataFrame.")

#Column Deletion for output_2.csv:
csv_file_path = '/content/output_2.csv'
df = pd.read_csv(csv_file_path)
column_to_delete = 'Class Name_left'
if column_to_delete in df.columns:
    df = df.drop(columns=[column_to_delete])
    new_csv_file_path = 'output_2.csv'
    df.to_csv(new_csv_file_path, index=False)
    print(f"Column '{column_to_delete}' deleted. New data saved to {new_csv_file_path}")
else:
    print(f"Column '{column_to_delete}' does not exist in the DataFrame.")

#Merging output_1.csv and output_2.csv to ocr_result.csv:
csv_file_path1 = '/content/output_1.csv'
df1 = pd.read_csv(csv_file_path1,encoding='latin1')
csv_file_path2 = '/content/output_2.csv'
df2 = pd.read_csv(csv_file_path2,encoding='latin1')
# Merge the two DataFrames on the "Image File" column
merged_df = pd.merge(df1, df2, on='Image File', how='outer', suffixes=('_left', '_right'))
merged_df = merged_df.replace('\n', ' ', regex=True)

# Save the merged DataFrame to an Excel file
csv_file_path = 'ocr_result.csv'
merged_df.to_csv(csv_file_path, index=False)
print(f"Combined data saved to {csv_file_path}")
csv_file_path = '/content/ocr_result.csv'
df = pd.read_csv(csv_file_path)
column_to_delete = 'Class Name_left'
if column_to_delete in df.columns:
    df = df.drop(columns=[column_to_delete])
    new_csv_file_path = 'ocr_result.csv'
    df.to_csv(new_csv_file_path, index=False)
    print(f"Column '{column_to_delete}' deleted. New data saved to {new_csv_file_path}")
else:
    print(f"Column '{column_to_delete}' does not exist in the DataFrame.")

#Cleaning Symbols and Saving to train.csv
csv_file_path = '/content/ocr_result.csv'
df = pd.read_csv(csv_file_path)
# Define a function to clean symbols in each cell
def clean_symbols(cell_value):
    if isinstance(cell_value, str):
        cleaned_value = cell_value.replace('Ã¢ÂÂ', '').replace('Ã¢ÂÂ', '').replace('Ã¢ÂÂ',' ')
        return cleaned_value
    else:
        return cell_value
df_cleaned = df.applymap(clean_symbols)
new_csv_file_path = 'train.csv'
df_cleaned.to_csv(new_csv_file_path, index=False)
print(f"Unknown symbols removed. Cleaned data saved to {new_csv_file_path}")

#Combining Columns and Saving to train.csv Again
csv_file_path = 'train.csv'
df = pd.read_csv(csv_file_path)
def combine_columns(row):
    return f"{row['PSM11 RGB']} {row['PSM11 gray']} {row['PSM3 RGB']} {row['PSM3 Gray']}"
df['combined_column'] = df.apply(combine_columns, axis=1)
new_csv_file_path = 'train.csv'
df.to_csv(new_csv_file_path, index=False)
print(f"Columns combined. Modified data saved to {new_csv_file_path}")

Column 'Class Name_right' deleted. New data saved to output_1.csv
Column 'Class Name_left' deleted. New data saved to output_2.csv
Combined data saved to ocr_result.csv
Column 'Class Name_left' deleted. New data saved to ocr_result.csv
Unknown symbols removed. Cleaned data saved to train.csv
Columns combined. Modified data saved to train.csv


# **Data Processing Workflow**

In [6]:
#Combining Columns in OCR_test_combined.csv and Saving to test.csv
csv_file_path = '/content/OCR_test_combined.csv'
df = pd.read_csv(csv_file_path)
def combine_columns(row):
    return f"{row['PSM11 RGB']} {row['PSM11 Gray']} {row['PSM3 RGB']} {row['PSM3 Gray']}"
# Apply the combine_columns function to create a new 'combined_column'
df['combined_column'] = df.apply(combine_columns, axis=1)
new_csv_file_path = 'test.csv'
df.to_csv(new_csv_file_path, index=False)
print(f"Columns combined. Modified data saved to {new_csv_file_path}")

#Cleaning Column 'combined_column' in train.csv
csv_file_path = 'train.csv'
df = pd.read_csv(csv_file_path)
column_to_clean = 'combined_column'
# Define a function to keep only German characters and numbers
def keep_german_and_numbers(cell_value):
    cleaned_value = re.sub(r'[^a-zA-ZäöüÄÖ%=Üß0-9]', '  ', str(cell_value))
    return cleaned_value
df[column_to_clean] = df[column_to_clean].apply(keep_german_and_numbers)
new_csv_file_path = 'train.csv'
df.to_csv(new_csv_file_path, index=False)
print(f"Column '{column_to_clean}' cleaned. Modified data saved to {new_csv_file_path}")

#Cleaning Column 'combined_column' in test.csv
csv_file_path = 'test.csv'
df = pd.read_csv(csv_file_path)
column_to_clean = 'combined_column'
def keep_german_and_numbers(cell_value):
    cleaned_value = re.sub(r'[^a-zA-ZäöüÄÖ%=Üß0-9]', '  ', str(cell_value))
    return cleaned_value
# Apply the keep_german_and_numbers function to the specified column
df[column_to_clean] = df[column_to_clean].apply(keep_german_and_numbers)
new_csv_file_path = 'test.csv'
df.to_csv(new_csv_file_path, index=False)
print(f"Column '{column_to_clean}' cleaned. Modified data saved to {new_csv_file_path}")

#Merging train.csv and quantity_train.csv to train_final.csv
csv_file_path1 = '/content/train.csv'
df1 = pd.read_csv(csv_file_path1,encoding='latin1')
csv_file_path2 = '/content/quantity_train.csv'
df2 = pd.read_csv(csv_file_path2,encoding='latin1')
merged_df = pd.merge(df1, df2, on='Image File', how='outer', suffixes=('_left', '_right'))
merged_df = merged_df.replace('\n', ' ', regex=True)
csv_file_path = 'train_final.csv'
merged_df.to_csv(csv_file_path, index=False)
print(f"Combined data saved to {csv_file_path}")

#Merging test.csv and quantity_test.csv to test_final.csv
csv_file_path1 = '/content/test.csv'
df1 = pd.read_csv(csv_file_path1,encoding='latin1')
csv_file_path2 = '/content/quantity_test.csv'
df2 = pd.read_csv(csv_file_path2,encoding='latin1')
merged_df = pd.merge(df1, df2, on='Image File', how='outer', suffixes=('_left', '_right'))
merged_df = merged_df.replace('\n', ' ', regex=True)
csv_file_path = 'test_final.csv'
merged_df.to_csv(csv_file_path, index=False)
print(f"Combined data saved to {csv_file_path}")

#Column Deletion in train_final.csv
csv_file_path = 'train_final.csv'
df = pd.read_csv(csv_file_path)
columns_to_delete = ['PSM11 gray', 'PSM11 RGB', 'PSM3 RGB', 'PSM3 Gray']
df.drop(columns=columns_to_delete, inplace=True)
new_csv_file_path = 'train_final.csv'
df.to_csv(new_csv_file_path, index=False)
print(f"Columns {columns_to_delete} deleted. Modified data saved to {new_csv_file_path}")

#Column Deletion in test_final.csv
csv_file_path = 'test_final.csv'
df = pd.read_csv(csv_file_path)
columns_to_delete = ['PSM11 Gray', 'PSM11 RGB', 'PSM3 RGB', 'PSM3 Gray']
df.drop(columns=columns_to_delete, inplace=True)
new_csv_file_path = 'test.csv'
df.to_csv(new_csv_file_path, index=False)
print(f"Columns {columns_to_delete} deleted. Modified data saved to {new_csv_file_path}")

#Combining Columns in train_final.csv and Saving to train.csv
csv_file_path = 'train_final.csv'
df = pd.read_csv(csv_file_path)
def combine_columns(row):
    return f"{row['combined_column_left']} {row['combined_column_right']} "
df['combined_column'] = df.apply(combine_columns, axis=1)
new_csv_file_path = 'train_final.csv'
df.to_csv(new_csv_file_path, index=False)
print(f"Columns combined. Modified data saved to {new_csv_file_path}")

#Combining Columns in test.csv and Saving to test_final.csv
csv_file_path = 'test.csv'
df = pd.read_csv(csv_file_path)
def combine_columns(row):
    return f"{row['combined_column_left']} {row['combined_column_right']} "
df['combined_column'] = df.apply(combine_columns, axis=1)
new_csv_file_path = 'test_final.csv'
df.to_csv(new_csv_file_path, index=False)
print(f"Columns combined. Modified data saved to {new_csv_file_path}")
df.head()

Columns combined. Modified data saved to test.csv
Column 'combined_column' cleaned. Modified data saved to train.csv
Column 'combined_column' cleaned. Modified data saved to test.csv
Combined data saved to train_final.csv
Combined data saved to test_final.csv
Columns ['PSM11 gray', 'PSM11 RGB', 'PSM3 RGB', 'PSM3 Gray'] deleted. Modified data saved to train_final.csv
Columns ['PSM11 Gray', 'PSM11 RGB', 'PSM3 RGB', 'PSM3 Gray'] deleted. Modified data saved to test.csv
Columns combined. Modified data saved to train_final.csv
Columns combined. Modified data saved to test_final.csv


,Image File,Class Name,combined_column_left,combined_column_right,combined_column
0,8045.jpg,150,Oldesloer Weizenkorn 32 %Vol 11=...,"0,71 0,71 0,71 nan",Oldesloer Weizenkorn 32 %Vol 11=...
1,8054.jpg,150,oldesloer Weizenkorn 32% vol 9 ...,"9,7L 9,7L 0,7L nan",oldesloer Weizenkorn 32% vol 9 ...
2,8075.jpg,150,Oldesloer Weizenkorn Aus sonnen A ...,"8,56 W 0,71 8,56 W 0,71 0,71 8,56 l 0,71",Oldesloer Weizenkorn Aus sonnen A ...
3,8044.jpg,150,we 80510 S Weizen hoen 0 ...,0 L nan nan 985l,we 80510 S Weizen hoen 0 ...
4,8039.jpg,150,Weizenkorn Oldesloer 32% Vol 0 71...,"0,71 L 0,71 0,71 L 0,71 0,71 L 0,71 0,71 L 0,71",Weizenkorn Oldesloer 32% Vol 0 71...


# **Text Cleaning and Tokenization**

In [7]:
file_path = 'test_final.csv'
df = pd.read_csv(file_path)
column_name = 'combined_column'
def clean_and_tokenize(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('german'))
    tokens = [token for token in tokens if token not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens

df[column_name] = df[column_name].apply(clean_and_tokenize)
output_file_path = 'test_final.csv'
df.to_csv(output_file_path, index=False)

file_path = 'test_final.csv'
df = pd.read_csv(file_path)
column_name = 'combined_column'
def clean_and_tokenize(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('german'))
    tokens = [token for token in tokens if token not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens

df[column_name] = df[column_name].apply(clean_and_tokenize)
output_file_path = 'test_final.csv'
df.to_csv(output_file_path, index=False)


df = pd.read_csv('train_final.csv')
column_name = 'combined_column'  # Replace 'your_column_name' with the actual column name
df[column_name] = df[column_name].str.replace(r'\bnan\b', '', regex=True)
# Remove specific words from the specified column
specific_words = ['es', 'un', 'ae','ann' 'se','pr','ni','se' 'an','nan','=','e','za','aaa','bbb','cc','de','ee','aa','ve','Fu','Ba','en','ii','een','naar','su','ut','ori','pn','"',]
for word in specific_words:
    df[column_name] = df[column_name].str.replace(r'\b{}\b'.format(word), '', regex=True)
df.to_csv('train_final.csv', index=False)


df = pd.read_csv('test_final.csv')
column_name = 'combined_column'  # Replace 'your_column_name' with the actual column name
df[column_name] = df[column_name].str.replace(r'\bnan\b', '', regex=True)
# Remove specific words from the specified column
specific_words = ['es', 'un', 'ae','ann' 'se','pr','ni','se' 'an','nan','=','e','za','aaa','bbb','cc','de','ee','aa','ve','Fu','Ba','en','ii','een','naar','su','ut','ori','pn','"',]
for word in specific_words:
    df[column_name] = df[column_name].str.replace(r'\b{}\b'.format(word), '', regex=True)
df.to_csv('test_final.csv', index=False)

letters_list = [char for char in string.ascii_lowercase]



**Data Processing and Cleaning Script**

In [8]:
df = pd.read_csv('test_final.csv')
# Specify the column from which you want to remove words
column_name = 'combined_column'  # Replace 'your_column_name' with the actual column name
# Remove "nan" from the specified column
df[column_name] = df[column_name].str.replace(r'\bnan\b', '', regex=True)
# Remove specific words from the specified column
specific_words = letters_list
for word in specific_words:
    df[column_name] = df[column_name].str.replace(r'\b{}\b'.format(word), '', regex=True)
# Write the modified DataFrame back to a CSV file
df.to_csv('test_final.csv', index=False)

df = pd.read_csv('train_final.csv')
# Specify the column from which you want to remove words
column_name = 'combined_column'  # Replace 'your_column_name' with the actual column name
# Remove "nan" from the specified column
df[column_name] = df[column_name].str.replace(r'\bnan\b', '', regex=True)
# Remove specific words from the specified column
specific_words = letters_list
for word in specific_words:
    df[column_name] = df[column_name].str.replace(r'\b{}\b'.format(word), '', regex=True)
# Write the modified DataFrame back to a CSV file
df.to_csv('train_final.csv', index=False)

csv_file_path = 'test_final.csv'
df = pd.read_csv(csv_file_path)
column_with_nan = 'Class Name'
# Drop rows where the specified column has NaN values
df = df.dropna(subset=[column_with_nan])
# Save the modified DataFrame to a new CSV file
new_csv_file_path = 'test_final.csv'
df.to_csv(new_csv_file_path, index=False)
print(f"Rows with NaN values in column '{column_with_nan}' deleted. Modified data saved to {new_csv_file_path}")

csv_file_path = 'train_final.csv'
df = pd.read_csv(csv_file_path)
column_with_nan = 'Class Name_right_left'
# Drop rows where the specified column has NaN values
df = df.dropna(subset=[column_with_nan])
# Save the modified DataFrame to a new CSV file
new_csv_file_path = 'train_final.csv'
df.to_csv(new_csv_file_path, index=False)
print(f"Rows with NaN values in column '{column_with_nan}' deleted. Modified data saved to {new_csv_file_path}")
df.head()

Rows with NaN values in column 'Class Name' deleted. Modified data saved to test_final.csv
Rows with NaN values in column 'Class Name_right_left' deleted. Modified data saved to train_final.csv


,Image File,Class Name_right_left,combined_column_left,Class Name_right_right,combined_column_right,combined_column
0,8047.jpg,150.0,e Oldesloer Weizenkorn 32% ...,NaN,NaN,Oldesloer Weizenkorn 32% ...
1,8061.jpg,150.0,PN Weizenkorn 32% vol Be je 0 ...,NaN,NaN,PN Weizenkorn 32% vol Be je 0 ...
2,8038.jpg,150.0,Oldesloer Weizenkorn 2 ol ori ...,NaN,NaN,Oldesloer Weizenkorn 2 ol ...
3,8057.jpg,150.0,Oldesloher Weizenkorn 32% vol 0 7 ...,NaN,NaN,Oldesloher Weizenkorn 32% vol 0 7 ...
4,8037.jpg,150.0,Oldesloer Weizenkorn 32% vol 0 7Lit...,NaN,NaN,Oldesloer Weizenkorn 32% vol 0 7Lit...


# **Data Preparation and TF-IDF Vectorization**

In [9]:
df=pd.read_csv('train_final.csv')
x_train_data=df['combined_column']
label=df['Class Name_right_left'].astype(int)
df_test =pd.read_csv('test_final.csv')
x_validation_data=df_test['combined_column']
label_validation=df_test['Class Name']

x_train_data.fillna('', inplace=True)
# Create and fit the TfidfVectorizer
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(x_train_data)
# Transform the data
x_train_vec = tfidf_vect.transform(x_train_data)
# Replace NaN values with an empty string
x_validation_data.fillna('', inplace=True)

tfidf_vect=TfidfVectorizer()
tfidf_vect.fit(x_train_data)
x_train_vec = tfidf_vect.transform(x_train_data)
x_validation_vec = tfidf_vect.transform(x_validation_data)


# **Training the Multinomial Naive Bayes Model**

In [10]:
nb_classifier = MultinomialNB()
nb_classifier.fit(x_train_vec, label[:])
# Prediction
y_pred = nb_classifier.predict(x_validation_vec)
# Accuracy
accuracy = accuracy_score(label_validation, y_pred)
print("Multinomial Naive Bayes Accuracy:", accuracy)

Multinomial Naive Bayes Accuracy: 0.8457932692307693


# **Saving the Model and TF-IDF Vectors**

In [13]:
# Save the trained model
joblib.dump(nb_classifier, 'multinomial_naive_bayes_model.joblib')

# Save the TF-IDF vectors
with open('tfidf_vectors.pkl', 'wb') as f:
    pickle.dump(tfidf_vect, f)
